In [1]:
import struct

def float_to_binary(value):
    # 使用 struct.pack 将浮点数转换为字节对象
    # 将字节对象转换为二进制字符串
    binary = ''.join(format(b, '08b') for b in bytearray(struct.pack('>e', value)))
    # 提取符号位、指数位和尾数位
  
    sign = str(binary[0])
    if (binary[1:6] == '00000' and binary[6:] == '0' * 10):
        sign = '0'  
        exponent = '10000'
        mantissa = '0' + binary[6:]
    else:
        sign = str(binary[0])
        exponent = binary[1:6]
        mantissa = "1" + binary[6:]

    return sign, exponent, mantissa

def shift_exponent(binary):
    if binary.startswith("1"):
        shift = '0' * 5 + binary[1:5]
    else:
        shift = '1' * 5 + binary[1:5]

    return shift

def shift_back_exponent(binary):
    if binary[0] == "1":
        shift = '0' + binary[5:9].zfill(4)
    elif binary[0] == "0":
        shift = '1' + binary[5:9].zfill(4)
    return shift

def binary_to_binary_complement16(signal, binary):
    if (signal == '1'):
        #按位取反
        inverted = "".join("1" if bit == "0" else "0" for bit in binary)
        #最后一位加一
        complement = str(bin(int(inverted, 2))[2:].zfill(16))[-16:]  # 加1得到补码
        mantissa = signal + signal + complement
    else:
        mantissa = signal + signal + binary
    return mantissa

def binary_to_float(binary):
    padded_binary = binary.zfill(16)
    byte_str = bytes([int(binary[i:i+8], 2) for i in range(0, len(padded_binary), 8)])
    # 使用 struct 模块将字节串转换为浮点数
    value = struct.unpack('>e', byte_str)[0]
    return value

def Exponent_MUlTI(a_exponent, b_exponent):
    a_exponent = a_exponent[0] + a_exponent
    b_exponent = b_exponent[0] + b_exponent
    ab_exponent = str(bin(int(a_exponent, 2) + int(b_exponent, 2))[2:]).zfill(9)[-9:]
    return ab_exponent

def exponent_judge(exponent, mantissa):
    if exponent[3:5] == '01' or exponent[5:9] == '1111': #上溢
        exponent = '011111110'
        if mantissa.startswith('0'):
            mantissa = '0' + '1' * 25
        else:
            mantissa = '1' + '00000000000'*11 + '1'*14
    elif exponent[3:5] == '10': #下溢
        exponent = '100000000'
        mantissa = '0' * 26
    else:
        exponent = exponent
        mantissa = mantissa

    return exponent, mantissa

def INPUT (float):
    sign, exponent, mantissa = float_to_binary(float)
    exponent = shift_exponent(exponent)
    return  sign, exponent, mantissa

def MULTI(a_sign, a_mantissa, b_sign, b_mantissa):
    ab_sign = str(int(a_sign, 2) ^ int(b_sign, 2))
    ab_mantissa = str(bin(int(a_mantissa, 2) * int(b_mantissa, 2))[2:].zfill(16))   # 乘法
    ab_mantissa = binary_to_binary_complement16(ab_sign, ab_mantissa)               # 转补码 因为要将结果拼接所以不能舍入处理
    return ab_mantissa

def Splicing_PE (Exp_in, low_mantissa, high_mantissa, model):
    if (model == '1'):
        if(Exp_in == '1'):
            low_mantissa =  low_mantissa[0] * 7 + low_mantissa + low_mantissa[0] 
        else:
            low_mantissa =  low_mantissa[0] * 8 + low_mantissa     
    else:
        low_mantissa = low_mantissa + low_mantissa[0] * 8
        
    S_mantissa = str(bin(int(low_mantissa[0:18], 2) + int(high_mantissa, 2))[2:].zfill(18))[-18:]
    S_mantissa = S_mantissa + low_mantissa[18:26]
    Exp = '0'


    if S_mantissa.startswith('01'):
        S_mantissa = '0' + S_mantissa[:-1]
        Exp = '1'
    elif S_mantissa.startswith('10'):
        S_mantissa = '1' + S_mantissa[:-1]
        Exp = '1'

    return Exp, S_mantissa


In [2]:
#与两个尾数的移位有关系，所以可能要修改 
def exponent_judge_16(exponent, mantissa):
    if exponent.startswith('01') or exponent[2:] == '1111': #上溢
        exponent = '011110'
        if mantissa.startswith('0'):
            mantissa = '0' + '1' * 12
        else:
            mantissa = '1' + '00000000000'*11 + '1'
    elif exponent.startswith('10'): #下溢
        exponent = '100000'
        mantissa = '0' * 13
    else:
        exponent = exponent
        mantissa = mantissa

    return exponent, mantissa

def shift_exponent_16(binary):
    if binary.startswith("1"):
        shift = '0' + binary[1:].zfill(4)
    else:
        shift = '1' + binary[1:].zfill(4)
    return shift

def binary_to_float_16(binary):
    padded_binary = binary.zfill(16)
    byte_string = bytes(int(padded_binary[i:i+8], 2) for i in range(0, 16, 8))
    value = struct.unpack('>e', byte_string)[0]
    return value

def binary_to_binary_complement_16(signal, binary):
    if (signal == '1'):
        #按位取反
        inverted = "".join("1" if bit == "0" else "0" for bit in binary)
        complement = str(bin(int(inverted, 2) + 1)[2:].zfill(11))[-11:]  
        mantissa = signal + signal + complement
    else:
        mantissa = signal + signal + binary
    return mantissa

def Exponent_Shift_16(a_exponent, b_exponent):
    #这里判断的条件要改
    if a_exponent.startswith('10') :
        if b_exponent.startswith('10'): #这里对应的下溢出
            max_exponent = '100000'
            max_signal = '00'
            shift_exponent = '0'
            overflow = '11'
        else:
            max_exponent = b_exponent
            shift_exponent = '0'
            max_signal = '10'
            overflow = '01'
    elif b_exponent.startswith('10'):
        max_exponent = a_exponent
        max_signal = '01'
        shift_exponent = '0'
        overflow = '10'
    else:
        overflow = '00'
        #因为python笨比的计算机制，所以正数和负数参与的计算的结果不一样，但是实际FPGA中是一样的。
        if(a_exponent.startswith('1') and b_exponent.startswith('0')):
            shift_exponent = str(bin(int(b_exponent, 2) - int(a_exponent, 2)))
        else:
            shift_exponent = str(bin(int(a_exponent, 2) - int(b_exponent, 2)))
        
        if(a_exponent[0] == b_exponent[0]): 
            if(a_exponent[0] == '0'): 
                #这里是被迫采用这种判断方法的，因为python无法计算二进制正数的补码运算
                if shift_exponent.startswith('-'):
                    shift_exponent = str(shift_exponent[3:].rjust(6, '0'))
                    if shift_exponent[1:6] == '00000':
                        shift_exponent = '000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = b_exponent  
                        max_signal = '10'          
                else:
                    shift_exponent = str(shift_exponent[2:].zfill(6))
                    if shift_exponent[1:6] == '00000':
                        shift_exponent = '000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = a_exponent
                        max_signal = '01'
                
                shift_exponent = int(shift_exponent, 2)

            else:
                if shift_exponent.startswith('-'):
                    shift_exponent = str(shift_exponent[3:].rjust(6, '1'))
                    if shift_exponent[1:6] == '00000':
                        shift_exponent = '000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = a_exponent  
                        max_signal = '01'          
                else:
                    shift_exponent = str(shift_exponent[2:].zfill(6))
                    if shift_exponent[1:6] == '00000':
                        shift_exponent = '000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = b_exponent
                        max_signal = '10'

                if shift_exponent.startswith('1'):
                    inverted = "".join("1" if bit == "0" else "0" for bit in shift_exponent[1:6])
                    complement = str(bin(int(inverted, 2) + 1)[2:].zfill(5))[-5:]  # 加1得到补码
                    shift_exponent = '0' + complement
                shift_exponent = int(shift_exponent, 2)
        else:
            if(shift_exponent.startswith('-')):
                shift_exponent = str(shift_exponent[3:].rjust(6, '1'))
            else:
                shift_exponent = str(shift_exponent[2:].zfill(6))

            if(a_exponent[0] == '0'):
                max_exponent = a_exponent
                max_signal = '01'
            else:
                max_exponent = b_exponent
                max_signal = '10'

            if shift_exponent.startswith('1'):
                    inverted = "".join("1" if bit == "0" else "0" for bit in shift_exponent[1:6])
                    complement = str(bin(int(inverted, 2) + 1)[2:].zfill(5))[-5:]  # 加1得到补码
                    shift_exponent = '0' + complement
            shift_exponent = int(shift_exponent, 2)
    return shift_exponent, max_exponent, max_signal, overflow

def ADDER_16(Proga, Progb, a_mantissa, b_mantissa, shift_exponent, max_signal, overflow):
    if(overflow == '11'):
        ab_mantissa = '0' * 13 #修改
        Prog = '0'
        Inst = '11' #跳过
    elif (overflow == '01'):
        ab_mantissa = b_mantissa
        Prog = Progb
        Inst = '11' #跳过
    elif (overflow == '10'):
        ab_mantissa = a_mantissa
        Prog = Proga
        Inst = '11' #跳过
    else:
        if(max_signal == '00'):#A B相同的半区
            if int(Proga, 2) == int(Progb, 2):
                Prog = Proga
                shift = str(int(0))
                Inst = '00' #同加
            elif int(Proga, 2) < int(Progb, 2):
                Prog = Progb
                shift = str(int(Progb, 2) - int(Proga, 2))
                Inst = '01' #A对位
            else:
                Prog = Proga
                shift = str(int(Proga, 2) - int(Progb, 2))
                Inst = '10' #B对位
        elif(max_signal == '01'): #A大的半区
            shift = str(int(Proga, 2) + int(shift_exponent))
            if int(shift) == int(Progb, 2):
                Prog = Proga
                shift = str(int(0))
                Inst = '00' #同加
            elif int(shift) < int(Progb, 2):
                Prog = str(bin(int(Proga ,2) + 1)[2:])
                shift = str(int(1))#S
                Inst = '01' #A对位 
            else:
                Prog = Proga
                shift = str(int(shift) - int(Progb, 2))
                Inst = '10' #B对位
        elif(max_signal == '10'): #B大的半区
            shift = str(int(Progb, 2) + int(shift_exponent))
            if int(shift) == int(Proga, 2):
                Prog = Progb
                shift = str(int(0))
                Inst = '00' #同加
            elif int(shift) < int(Proga, 2):
                Prog = str(bin(int(Progb ,2) + 1)[2:])
                shift = str(int(1))#S
                Inst = '10' #B对位
            else:
                Prog = Progb
                shift = str(int(shift) - int(Proga, 2))
                Inst = '01' #A对位
        else:
            shift = '0'
            Inst ='11'
    
    if Inst == '00': # 同为加
        a = a_mantissa
        b = b_mantissa
        Prog_out = Prog
        ab_mantissa = str(bin(int(a, 2) + int(b, 2))[2:].zfill(13))[-13:]
    elif Inst == '01': # A对位 B不动
        if(int(shift) < 14 ):
            a = a_mantissa[0] * int(shift) + a_mantissa[0:13-int(shift)]
        else:
            a = a_mantissa[0] * 13
        b = b_mantissa
        Prog_out = Prog
        ab_mantissa = str(bin(int(a, 2) + int(b, 2))[2:].zfill(13))[-13:]
    elif Inst == '10': # B对位 A不动
        if(int(shift) < 14 ):
            b = b_mantissa[0] * int(shift) + b_mantissa[0:13-int(shift)]
        else:
            b = b_mantissa[0] * 13
        a = a_mantissa
        Prog_out = Prog
        ab_mantissa = str(bin(int(a, 2) + int(b, 2))[2:].zfill(13))[-13:]
    else:
        Prog_out = Prog
        ab_mantissa = ab_mantissa

    if ab_mantissa.startswith('01'):
        ab_mantissa = ab_mantissa[0] + ab_mantissa[0:12]
        Prog_out = str(bin(int(Prog_out) + 1)[2:])
    elif ab_mantissa.startswith('10'):
        ab_mantissa = ab_mantissa[0] + ab_mantissa[0:12]
        Prog_out = str(bin(int(Prog_out) + 1)[2:])

    return Prog_out, ab_mantissa

def OUTPUT_16(sign_result, exponent_result, mantissa_result):
    #需要增加一个计算00停止的条件
    while mantissa_result[0:3] == "000" or mantissa_result[0:3] == "111":
        mantissa_result = str(mantissa_result[1:13]) + '0'  # 数据左移 指数减少
        exponent_result = str(bin((int(exponent_result, 2) - 1) & 0x3F))[2:].zfill(6) # 指数位减 1
        if(mantissa_result == '1' * 13 or mantissa_result == '0' * 13):
            exponent_result = '100000'
            break
    while mantissa_result[0:2] == "01" or mantissa_result[0:2] == "10":
        mantissa_result = mantissa_result[0] + mantissa_result[0:12] # 数据右移 指数增加
        exponent_result = str(bin((int(exponent_result, 2) + 1) & 0x3F))[2:].zfill(6) # 指数位加 1

    #需增加一个判断负0的条件
    exponent_result, mantissa_result = exponent_judge_16(exponent_result, mantissa_result)
    #指数转换
    if exponent_result[0] == '1':
        exponent_result = '0' + exponent_result[2:6]
    else:
        exponent_result = '1' + exponent_result[2:6]

    #补码转原码
    if sign_result == '1':
        # 对补码的符号位取反
        inverted = "".join("1" if bit == "0" else "0" for bit in mantissa_result)  
        # 加1得到原码
        mantissa_result = bin(int(inverted, 2) + 1)[2:].zfill(13)
    else:
        mantissa_result = mantissa_result

    result = str(sign_result + exponent_result + mantissa_result[3: 13])
    return result

def IEEE754_ADD_16(fA, fB):
    # 改
    A1_exponent = shift_exponent_16(fA[1:6])
    if (fA[1:] ==  '0' * 15):
        A1_sign = '0'
        A1_mantissa = '0' + fA[6:]
    else:
        A1_sign = str(fA[0])
        A1_mantissa = "1" + fA[6:]
    A1_mantissa = binary_to_binary_complement_16(A1_sign, A1_mantissa)
    # 改
    B1_exponent = shift_exponent_16(fB[1:6])
    if (fB[1:] == '0' * 15):
        B1_sign = '0'
        B1_mantissa = '0' + fB[6:]
    else:
        B1_sign = str(fB[0])
        B1_mantissa = "1" + fB[6:]
    B1_mantissa = binary_to_binary_complement_16(B1_sign, B1_mantissa)

    Shift, Exponent, max_signal, overflow = Exponent_Shift_16(A1_exponent, B1_exponent)

    Prog, Mantissa = ADDER_16('0', '0', A1_mantissa, B1_mantissa, Shift, max_signal, overflow)

    Signal = Mantissa[0]
    Exponent = str(bin(int(Exponent, 2) + int(Prog, 2))[2:].zfill(6))

    result = OUTPUT_16(Signal, Exponent, Mantissa)
    return result

def IEEE754_MUL_16(fA, fB):
    # 改
    A1_exponent = shift_exponent_16(fA[1:6])
    if (fA[1:6] == '00000' and fA[6:] == '0' * 10):
        A1_sign = '0'
        A1_mantissa = '0' + fA[6:]
    else:
        A1_sign = str(fA[0])
        A1_mantissa = "1" + fA[6:]

    # 改
    B1_exponent = shift_exponent_16(fB[1:6])
    if (fB[1:6] == '00000' and fB[6:] == '0' * 10):
        B1_sign = '0'
        B1_mantissa = '0' + fB[6:]
    else:
        B1_sign = str(fB[0])
        B1_mantissa = "1" + fB[6:]

    A1_exponent = A1_exponent[0] + A1_exponent
    B1_exponent = B1_exponent[0] + B1_exponent

    Exponent = str(bin(int(A1_exponent, 2) + int(B1_exponent, 2))[2:]).zfill(6)[-6:]
    Exponent = str(bin((int(Exponent, 2) + 1) & 0x3F))[2:].zfill(6) # 指数位加 1
    Exponent = str(bin((int(Exponent, 2) + 1) & 0x3F))[2:].zfill(6) # 指数位加 1 

    # 改 位数21 11*11 = 22-1 =21
    ab_sign = str(int(A1_sign, 2) ^ int(B1_sign, 2))
    ab_mantissa = str(bin(int(A1_mantissa, 2) * int(B1_mantissa, 2))[2:].zfill(22))   # 乘法

    Mantissa = binary_to_binary_complement_16(ab_sign, ab_mantissa[0:11]) 
    Signal = Mantissa[0]

    result = OUTPUT_16(Signal, Exponent, Mantissa)
    return result


In [3]:
#输入
floatA1 = -49848.83558772618
floatB1 = -22854.162689145945
floatA2 = 30108.96249318197
floatB2 = 56235.51401716222
floatA3 = -2451.0661169491505
floatB3 = 3032.0082521096774

A1_sign, A1_exponent, A1_mantissa = INPUT(floatA1)
B1_sign, B1_exponent, B1_mantissa = INPUT(floatB1)
#更改了数据填充
A1H = A1_mantissa[0:8]
A1L = A1_mantissa[8:11] + '0' * 5

B1H = B1_mantissa[0:8]
B1L = B1_mantissa[8:11] + '0' * 5

print('A1H = {}'.format(A1H))
print('A1L = {}'.format(A1L))
print('B1H = {}'.format(B1H))
print('B1L = {}'.format(B1L))

A2_sign, A2_exponent, A2_mantissa = INPUT(floatA2)
B2_sign, B2_exponent, B2_mantissa = INPUT(floatB2)

A2H = A2_mantissa[0:8]
A2L = A2_mantissa[8:11] + '0' * 5

B2H = B2_mantissa[0:8]
B2L = B2_mantissa[8:11] + '0' * 5

A3_sign, A3_exponent, A3_mantissa = INPUT(floatA3)
B3_sign, B3_exponent, B3_mantissa = INPUT(floatB3)

A3H = A3_mantissa[0:8]
A3L = A3_mantissa[8:11] + '0' * 5

B3H = B3_mantissa[0:8]
B3L = B3_mantissa[8:11] + '0' * 5

A1H = 11000010
A1L = 11000000
B1H = 10110010
B1L = 10000000


In [4]:
#相当于每一个PE内部的数据进行处理，相当于硬件中把每一个PE内部的数据的都求出来
#第一组
A1LBH_mantissa = MULTI(A1_sign, A1L, B1_sign, B1H)
A1HBL_mantissa = MULTI(A1_sign, A1H, B1_sign, B1L)
A1HBH_mantissa = MULTI(A1_sign, A1H, B1_sign, B1H)
#第二组
A2LBH_mantissa = MULTI(A2_sign, A2L, B2_sign, B2H)
A2HBL_mantissa = MULTI(A2_sign, A2H, B2_sign, B2L)
A2HBH_mantissa = MULTI(A2_sign, A2H, B2_sign, B2H)
#第三组
A3LBH_mantissa = MULTI(A3_sign, A3L, B3_sign, B3H)
A3HBL_mantissa = MULTI(A3_sign, A3H, B3_sign, B3L)
A3HBH_mantissa = MULTI(A3_sign, A3H, B3_sign, B3H)

print('A1LBH_mantissa = {}'.format(A1LBH_mantissa))
print('A1HBL_mantissa = {}'.format(A1HBL_mantissa))
print('A1HBH_mantissa = {}'.format(A1HBH_mantissa))

print('A2LBH_mantissa = {}'.format(A2LBH_mantissa))
print('A2HBL_mantissa = {}'.format(A2HBL_mantissa))
print('A2HBH_mantissa = {}'.format(A2HBH_mantissa))

print('A3LBH_mantissa = {}'.format(A3LBH_mantissa))
print('A3HBL_mantissa = {}'.format(A3HBL_mantissa))
print('A3HBH_mantissa = {}'.format(A3HBH_mantissa))

#PE 纵列拼接 
#FP32的3*3的纵列拼接在这里被压缩为一个3*1的纵列矩阵
#相当于PE内部的计算结果逐步累加，最终传播的到下层的ADDER的这个过程。
#第一组
Exp1H, A1HBLM_mantissa = Splicing_PE('0'  , A1HBL_mantissa,  A1LBH_mantissa, '0') #12
Exp1H, A1HBMH_mantissa = Splicing_PE(Exp1H, A1HBLM_mantissa, A1HBH_mantissa, '1') #23
#第二组
Exp2H, A2HBLM_mantissa = Splicing_PE('0'  , A2HBL_mantissa,  A2LBH_mantissa, '0') #12
Exp2H, A2HBMH_mantissa = Splicing_PE(Exp2H, A2HBLM_mantissa, A2HBH_mantissa, '1') #23
#第三组
Exp3H, A3HBLM_mantissa = Splicing_PE('0'  , A3HBL_mantissa,  A3LBH_mantissa, '0') #12
Exp3H, A3HBMH_mantissa = Splicing_PE(Exp3H, A3HBLM_mantissa, A3HBH_mantissa, '1') #23

print('Exp1H = {}'.format(Exp1H))
print('Exp2H = {}'.format(Exp2H))
print('Exp3H = {}'.format(Exp3H))
print('A1HBMH_mantissa = {}'.format(A1HBMH_mantissa))
print('A2HBMH_mantissa = {}'.format(A2HBMH_mantissa))
print('A3HBMH_mantissa = {}'.format(A3HBMH_mantissa))

A1LBH_mantissa = 001000010110000000
A1HBL_mantissa = 000110000100000000
A1HBH_mantissa = 001000011011100100
A2LBH_mantissa = 000011011011000000
A2HBL_mantissa = 001001001011100000
A2HBH_mantissa = 001100100100001001
A3LBH_mantissa = 111101000010111111
A3HBL_mantissa = 111011001101111111
A3HBH_mantissa = 111000111100001010
Exp1H = 0
Exp2H = 0
Exp3H = 0
A1HBMH_mantissa = 00100001111100101010000000
A2HBMH_mantissa = 00110010011101001010100000
A3HBMH_mantissa = 11100011101000111000111110


In [5]:
def Exponent_Shift(a_exponent, b_exponent):
    if a_exponent[3:5] =='10':
        if b_exponent[3:5] == '10': #这里对应的下溢出
            max_exponent = '111100000'
            max_signal = '00'
            shift_exponent = '0'
            overflow = '11'
        else:
            max_exponent = b_exponent
            shift_exponent = '0'
            max_signal = '01'
            overflow = '01'
    elif b_exponent[3:5] == '10':
        max_exponent = a_exponent
        max_signal = '10'
        shift_exponent = '0'
        overflow = '10'
    else:
        overflow = '00'
        #因为python笨比的计算机制，所以正数和负数参与的计算的结果不一样，但是实际FPGA中是一样的。
        if(a_exponent.startswith('1') and b_exponent.startswith('0')):
            shift_exponent = str(bin(int(b_exponent, 2) - int(a_exponent, 2)))
        else:
            shift_exponent = str(bin(int(a_exponent, 2) - int(b_exponent, 2)))

        if(a_exponent[0] == b_exponent[0]): 
            if(a_exponent[0] == '0'): 
                #这里是被迫采用这种判断方法的，因为python无法计算二进制正数的补码运算
                if shift_exponent.startswith('-'):
                    shift_exponent = str(shift_exponent[3:].rjust(9, '0'))
                    if shift_exponent[1:9] == '00000000':
                        shift_exponent = '000000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = b_exponent  
                        max_signal = '10'          
                else:
                    shift_exponent = str(shift_exponent[2:].zfill(9))
                    if shift_exponent[1:9] == '00000000':
                        shift_exponent = '000000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = a_exponent
                        max_signal = '01'
                
                shift_exponent = int(shift_exponent, 2)

            else:
                if shift_exponent.startswith('-'):
                    shift_exponent = str(shift_exponent[3:].rjust(9, '1'))
                    if shift_exponent[1:9] == '00000000':
                        shift_exponent = '000000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = a_exponent  
                        max_signal = '01'          
                else:
                    shift_exponent = str(shift_exponent[2:].zfill(9))
                    if shift_exponent[1:9] == '00000000':
                        shift_exponent = '000000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = b_exponent
                        max_signal = '10'

                if shift_exponent.startswith('1'):
                    inverted = "".join("1" if bit == "0" else "0" for bit in shift_exponent[1:9])
                    complement = str(bin(int(inverted, 2) + 1)[2:].zfill(8))[-8:]  # 加1得到补码
                    shift_exponent = '0' + complement
                shift_exponent = int(shift_exponent, 2)
        else:
            if(shift_exponent.startswith('-')):
                shift_exponent = str(shift_exponent[3:].rjust(9, '1'))
            else:
                shift_exponent = str(shift_exponent[2:].zfill(9))

            if(a_exponent[0] == '0'):
                max_exponent = a_exponent
                max_signal = '01'
            else:
                max_exponent = b_exponent
                max_signal = '10'

            if shift_exponent.startswith('1'):
                    inverted = "".join("1" if bit == "0" else "0" for bit in shift_exponent[1:9])
                    complement = str(bin(int(inverted, 2) + 1)[2:].zfill(8))[-8:]  # 加1得到补码
                    shift_exponent = '0' + complement
            shift_exponent = int(shift_exponent, 2)
    return shift_exponent, max_exponent, max_signal, overflow

In [6]:
print('A1_exponent= {}'.format(A1_exponent))
print('B1_exponent= {}'.format(B1_exponent))
print('A2_exponent= {}'.format(A2_exponent))
print('B2_exponent= {}'.format(B2_exponent))
print('A3_exponent= {}'.format(A3_exponent))
print('B3_exponent= {}'.format(B3_exponent))
#Exponent阵列，每一个PE先计算出每一个PE对应的
#A1
A1B_exponent = Exponent_MUlTI(A1_exponent, B1_exponent)
#A2
A2B_exponent = Exponent_MUlTI(A2_exponent, B2_exponent)
#A3
A3B_exponent = Exponent_MUlTI(A3_exponent, B3_exponent)
print('A1B_exponent = {}'.format(A1B_exponent))
print('A2B_exponent = {}'.format(A2B_exponent))
print('A3B_exponent = {}'.format(A3B_exponent))

Shift12, Exponent_12B, max_signal1, overflow1 = Exponent_Shift(A1B_exponent, A2B_exponent)
Shift23, Exponent_23B, max_signal2, overflow2 = Exponent_Shift(Exponent_12B, A3B_exponent)
print('Shift12      = {}'.format(Shift12))
print('Shift23      = {}'.format(Shift23))
print('Exponent_12B = {}'.format(Exponent_12B))
print('Exponent_23B = {}'.format(Exponent_23B))
print('max_signal1  = {}'.format(max_signal1))
print('max_signal2  = {}'.format(max_signal2))
print('overflow1    = {}'.format(overflow1))
print('overflow2    = {}'.format(overflow2))

A1_exponent= 000001110
B1_exponent= 000001101
A2_exponent= 000001101
B2_exponent= 000001110
A3_exponent= 000001010
B3_exponent= 000001010
A1B_exponent = 000011011
A2B_exponent = 000011011
A3B_exponent = 000010100
Shift12      = 0
Shift23      = 7
Exponent_12B = 000011011
Exponent_23B = 000011011
max_signal1  = 00
max_signal2  = 01
overflow1    = 00
overflow2    = 00


In [7]:
def ADDER(Proga, Progb, a_mantissa, b_mantissa, shift_exponent, max_signal, overflow):
    if(overflow == '11'):
        ab_mantissa = '0' * 26
        Prog = '0'
        Inst = '11' #跳过
    elif (overflow == '01'):
        ab_mantissa = b_mantissa
        Prog = Progb
        Inst = '11' #跳过
    elif (overflow == '10'):
        ab_mantissa = a_mantissa
        Prog = Proga
        Inst = '11' #跳过
    else:
        if(max_signal == '00'):#A B相同的半区
            if int(Proga, 2) == int(Progb, 2):
                Prog = Proga
                shift = str(int(0))
                Inst = '00' #同加
            elif int(Proga, 2) < int(Progb, 2):
                Prog = Progb
                shift = str(int(Progb, 2) - int(Proga, 2))
                Inst = '01' #A对位
            else:
                Prog = Proga
                shift = str(int(Proga, 2) - int(Progb, 2))
                Inst = '10' #B对位
        elif(max_signal == '01'): #A大的半区
            shift = str(int(Proga, 2) + int(shift_exponent))
            if int(shift) == int(Progb, 2):
                Prog = Proga
                shift = str(int(0))
                Inst = '00' #同加
            elif int(shift) < int(Progb, 2):
                Prog = str(bin(int(Proga ,2) + 1)[2:])
                shift = str(int(1))#S
                Inst = '01' #A对位 
            else:
                Prog = Proga
                shift = str(int(shift) - int(Progb, 2))
                Inst = '10' #B对位
        elif(max_signal == '10'): #B大的半区
            shift = str(int(Progb, 2) + int(shift_exponent))
            if int(shift) == int(Proga, 2):
                Prog = Progb
                shift = str(int(0))
                Inst = '00' #同加
            elif int(shift) < int(Proga, 2):
                Prog = str(bin(int(Progb ,2) + 1)[2:])
                shift = str(int(1))#S
                Inst = '10' #B对位
            else:
                Prog = Progb
                shift = str(int(shift) - int(Proga, 2))
                Inst = '01' #A对位
        else:
            shift = '0'
            Inst ='11'
    
    if Inst == '00': # 同为加
        a = a_mantissa
        b = b_mantissa
        Prog_out = Prog
        ab_mantissa = str(bin(int(a, 2) + int(b, 2))[2:].zfill(26))[-26:]
        print('a = {}'.format(a))
        print('b = {}'.format(b))
        # print('ab_mantissa = {}'.format(ab_mantissa))
    elif Inst == '01': # A对位 B不动
        if(int(shift) < 27 ):
            a = a_mantissa[0] * int(shift) + a_mantissa[0:26-int(shift)]
        else:
            a = a_mantissa[0] * 26
        b = b_mantissa
        Prog_out = Prog
        ab_mantissa = str(bin(int(a, 2) + int(b, 2))[2:].zfill(26))[-26:]
        print('a = {}'.format(a))
        print('b = {}'.format(b))
        # print('ab_mantissa = {}'.format(ab_mantissa))
    elif Inst == '10': # B对位 A不动
        if(int(shift) < 27 ):
            b = b_mantissa[0] * int(shift) + b_mantissa[0:26-int(shift)]
        else:
            b = b_mantissa[0] * 26
        a = a_mantissa
        Prog_out = Prog
        ab_mantissa = str(bin(int(a, 2) + int(b, 2))[2:].zfill(26))[-26:]
        print('a = {}'.format(a))
        print('b = {}'.format(b))
        # print('ab_mantissa = {}'.format(ab_mantissa))
    else:
        Prog_out = Prog
        ab_mantissa = ab_mantissa

    if ab_mantissa.startswith('01'):
        ab_mantissa = ab_mantissa[0] + ab_mantissa[0:25]
        Prog_out = str(bin(int(Prog_out) + 1)[2:])
    elif ab_mantissa.startswith('10'):
        ab_mantissa = ab_mantissa[0] + ab_mantissa[0:25]
        Prog_out = str(bin(int(Prog_out) + 1)[2:])

    return Prog_out, ab_mantissa


In [8]:
#Layer12
Prog12H, Mantissa12H = ADDER(Exp1H, Exp2H, A1HBMH_mantissa, A2HBMH_mantissa, Shift12, max_signal1, overflow1)

print('Prog12H = {}'.format(Prog12H))
print('Mantissa12H = {}'.format(Mantissa12H))
print('Prog3H = {}'.format(Exp3H))
print('A3HBMH_mantissa = {}'.format(A3HBMH_mantissa))

a = 00100001111100101010000000
b = 00110010011101001010100000
Prog12H = 1
Mantissa12H = 00101010001100111010010000
Prog3H = 0
A3HBMH_mantissa = 11100011101000111000111110


In [9]:
#Layer23
Prog23H, Mantissa23H = ADDER(Prog12H, Exp3H, Mantissa12H, A3HBMH_mantissa, Shift23, max_signal2, overflow2)
Mantissa23H = str(bin(int(Mantissa23H, 2) + 1)[2:].zfill(26))[-26:]
print('Prog23H = {}'.format(Prog23H))
print('Mantissa23H = {}'.format(Mantissa23H))
Prog = Prog23H
Mantissa = Mantissa23H

a = 00101010001100111010010000
b = 11111111111000111010001110
Prog23H = 1
Mantissa23H = 00101010000101110100011111


In [10]:
#指数位拼接
Signal = Mantissa[0]
Exponent = str(bin(int(Exponent_23B, 2) + int(Prog, 2))[2:].zfill(9))

print('Signal = {}'.format(Signal))
print('Exponent = {}'.format(Exponent))
print('Mantissa = {}'.format(Mantissa))


Signal = 0
Exponent = 000011100
Mantissa = 00101010000101110100011111


In [11]:
#754 Result
fA1 = ''.join(format(b, '08b') for b in bytearray(struct.pack('>e', floatA1))) #16
fB1 = ''.join(format(b, '08b') for b in bytearray(struct.pack('>e', floatB1))) #16
fA2 = ''.join(format(b, '08b') for b in bytearray(struct.pack('>e', floatA2))) #16
fB2 = ''.join(format(b, '08b') for b in bytearray(struct.pack('>e', floatB2))) #16
fA3 = ''.join(format(b, '08b') for b in bytearray(struct.pack('>e', floatA3))) #16
fB3 = ''.join(format(b, '08b') for b in bytearray(struct.pack('>e', floatB3))) #16

A1B1 = IEEE754_MUL_16(fA1, fB1)
A2B2 = IEEE754_MUL_16(fA2, fB2)
A3B3 = IEEE754_MUL_16(fA3, fB3)

AB12 = IEEE754_ADD_16(A1B1, A2B2)
AB23 = IEEE754_ADD_16(AB12, A3B3)

result_754 = binary_to_float_16(AB23)

print('result_754 = {}'.format(result_754))

#True Result

floatA1 = binary_to_float(fA1)
floatB1 = binary_to_float(fB1)
floatA2 = binary_to_float(fA2)
floatB2 = binary_to_float(fB2)
floatA3 = binary_to_float(fA3) 
floatB3 = binary_to_float(fB3)

print(floatA1)
print(floatB1)
print(floatA2)
print(floatB2)
print(floatA3)
print(floatB3)

#True Result
floatA1 = binary_to_float(fA1)
floatB1 = binary_to_float(fB1)
floatA2 = binary_to_float(fA2)
floatB2 = binary_to_float(fB2)
floatA3 = binary_to_float(fA3) 
floatB3 = binary_to_float(fB3)

TAB1 = floatA1 * floatB1
TAB2 = floatA2 * floatB2
TAB3 = floatA3 * floatB3

True_result = floatA1 * floatB1 + floatA2 * floatB2 + floatA3 * floatB3

if True_result > 65504:
    True_result = 65504
elif True_result < -65504:
    True_result = -65504

print(True_result)


result_754 = 0.0
-49856.0
-22848.0
30112.0
56224.0
-2452.0
3032.0
65504


In [12]:
def OUTPUT(sign_result, exponent_result, mantissa_result):
    while mantissa_result[0:3] == "000" or mantissa_result[0:3] == "111":
        mantissa_result = str(mantissa_result[1:26]) + '0'  # 数据左移 指数减少
        exponent_result = str(bin((int(exponent_result, 2) - 1) & 0x1FF))[2:].zfill(9) # 指数位减 1
        if(mantissa_result[0:13] == '1' * 13 or mantissa_result[0:13] == '0' * 13):
            exponent_result = '100000000'
            break

    while mantissa_result[0:2] == "01" or mantissa_result[0:2] == "10":
        mantissa_result = mantissa_result[0] + mantissa_result[0:25] # 数据右移 指数增加
        exponent_result = str(bin((int(exponent_result, 2) + 1) & 0x1FF))[2:].zfill(9) # 指数位加 1
    
    exponent_result = str(bin((int(exponent_result, 2) + 1) & 0x1FF))[2:].zfill(9) # 指数位加 1
    exponent_result = str(bin((int(exponent_result, 2) + 1) & 0x1FF))[2:].zfill(9) # 指数位加 1
    #结果的判断
    exponent_result, mantissa_result = exponent_judge(exponent_result, mantissa_result)

    #指数转换
    if exponent_result[0] == '1':
        exponent_result = '0' + exponent_result[5:9]
    else:
        exponent_result = '1' + exponent_result[5:9]

    #补码转原码
    if sign_result == '1':
        # 对补码的符号位取反
        inverted = "".join("1" if bit == "0" else "0" for bit in mantissa_result)  
        # 加1得到原码
        mantissa_result = bin(int(inverted, 2) + 1)[2:].zfill(26)
    else:
        mantissa_result = mantissa_result

    result = binary_to_float(sign_result + exponent_result + mantissa_result[3: 13])
    return result

result = OUTPUT(Signal, Exponent, Mantissa)

#结果对比
result_754_sign,  result_754_exponent,  result_754_mantissa   =  float_to_binary(result_754)
result_sign,      result_exponent,      result_mantissa       =  float_to_binary(result)

print('对比结果')
print(True_result)
print(result_754)
print(result)

Error_754 = abs(result - result_754)
Error_True = abs(result - True_result)
Error_inner = abs(result_754 - True_result)
precision_Ture = abs(result     - True_result )/abs(True_result + 0.0000000000001) * 100
precision_754  = abs(result     - result_754  )/abs(result_754  + 0.0000000000001) * 100
precision_inner= abs(result_754 - True_result )/abs(True_result + 0.0000000000001) * 100


print('Error_754        = {}'.format(Error_754))
print('precision_754    = {}'.format(precision_754))
print('Error_True       = {}'.format(Error_True))
print('precision_Ture   = {}'.format(precision_Ture))
print('Error_inner      = {}'.format(Error_inner))
print('precision_inner  = {}'.format(precision_inner))

对比结果
65504
0.0
65504.0
Error_754        = 65504.0
precision_754    = 6.5504e+19
Error_True       = 0.0
precision_Ture   = 0.0
Error_inner      = 65504.0
precision_inner  = 100.0
